<a href="https://colab.research.google.com/github/EnisBerk/speech_audio_understanding/blob/master/Classify_time%26location.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# GLOBAL parameters
# first run of notebook
first_run=True
data_required=True

In [0]:
if first_run==True:
  !pip install numpy scipy
  !pip install MulticoreTSNE


#   !pip install resampy tensorflow six
#   !pip install pysoundfile 
#   #from mp3 to wav
#   !pip install pydub



In [0]:
if first_run==True:
  import json
  import string
  import copy
  import random
  from os import listdir
  import pickle

  import numpy as np

  import string
  import re

  from sklearn.linear_model import SGDClassifier,LinearRegression
  from sklearn.model_selection import train_test_split
  from sklearn.pipeline import Pipeline
  from sklearn.linear_model import LogisticRegression
  from sklearn.feature_selection import SelectFromModel
  from sklearn.metrics import classification_report
  from sklearn.svm import LinearSVC
  from sklearn.naive_bayes import MultinomialNB
  from sklearn.tree import DecisionTreeClassifier
  from sklearn import tree
  from sklearn.ensemble import RandomForestClassifier
  from sklearn.neural_network import MLPClassifier
  from sklearn.ensemble import AdaBoostClassifier
  from sklearn.naive_bayes import GaussianNB
  from sklearn.feature_extraction.text import TfidfVectorizer

  from sklearn.feature_extraction.text import CountVectorizer
  from sklearn.feature_extraction.text import TfidfTransformer
  from sklearn.linear_model import SGDClassifier,LinearRegression
  from sklearn.model_selection import GridSearchCV

  from sklearn.metrics import accuracy_score


In [0]:
if first_run==True:
  from google.colab import auth
  auth.authenticate_user()


In [0]:
if data_required==True:
  !gsutil cp gs://deep_learning_enis/speech_audio_understanding/embeddings3.npy ./
  !gsutil cp gs://deep_learning_enis/speech_audio_understanding/postprocessed3.npy ./
  !gsutil cp gs://deep_learning_enis/speech_audio_understanding/data_file_indexes3.npy ./


Copying gs://deep_learning_enis/speech_audio_understanding/embeddings3.npy...
/ [1 files][ 46.9 MiB/ 46.9 MiB]                                                
Operation completed over 1 objects/46.9 MiB.                                     
Copying gs://deep_learning_enis/speech_audio_understanding/postprocessed3.npy...
/ [1 files][ 46.9 MiB/ 46.9 MiB]                                                
Operation completed over 1 objects/46.9 MiB.                                     
Copying gs://deep_learning_enis/speech_audio_understanding/data_file_indexes3.npy...
/ [1 files][973.2 KiB/973.2 KiB]                                                
Operation completed over 1 objects/973.2 KiB.                                    


In [0]:
embeddings=np.load('embeddings3.npy')
postprocessed=np.load('postprocessed3.npy')
data_file_indexes=np.load('data_file_indexes3.npy')
#get dict from numpy array
data_file_indexes=data_file_indexes[()]

In [0]:
mp3_data_files=sorted(list(data_file_indexes.keys()))

site_names=[]
hours=[]
for name in mp3_data_files:
  file_id=name
  name=name.split("_")
#   if len(name)!=18:
#     print(name)
  site_name=name[0]
  site_names.append(site_name)
  date=name[1]
  hour_min_sec=name[2]
  hour=hour_min_sec[0:2]
  hours.append(hour)
  year=date[0:4]
  month=date[4:6]
  day=date[6:8]
  site_id=name[2]

In [0]:
# print(len(set(hours)))
# print(len(set(site_names)))

# print(postprocessed.shape)
# print(embeddings.shape)
# print(len(site_names))
print(postprocessed[0].shape)

(128,)


In [0]:
# 10 second all together
postprocessed=postprocessed.reshape(9600,1280)

#average of 10 embeddings(1 second each)
embeddings_avg=np.mean(embeddings.reshape(10,-1,128),axis=0)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(
    postprocessed, hours, test_size=0.2, random_state=0)

In [0]:
classifiers_list=[LinearSVC,MultinomialNB,RandomForestClassifier,
                  AdaBoostClassifier,MLPClassifier,DecisionTreeClassifier,GaussianNB]

# classifiers_list=[MultinomialNB]

In [0]:
def run_experiment_w_features(X_train,Y_train,X_test,Y_test,theclassifiers):
#   theclassifiers=[MultinomialNB]

  if type(theclassifiers)!=list:
      theclassifiers=[theclassifiers]
  for theclassifier in theclassifiers:
  #       initialise classifier
      if theclassifier==LinearSVC:
          clf= theclassifier(max_iter=3000)
      else:
          clf= theclassifier()
  #     create model with tranining data
      model = clf.fit(X_train, Y_train)
  #     predict test set
      y_preds = model.predict(X_test)
      report = classification_report( y_test, y_preds )
      accuracy=accuracy_score( y_test, y_preds)
      print(str(theclassifier),accuracy)
      
#   return report, accuracy

In [0]:
run_experiment_w_features(X_train[:],y_train[:],X_test[:],y_test[:],classifiers_list)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 55.8M  100 55.8M    0     0   105M      0 --:--:-- --:--:-- --:--:--  105M


fatal: destination path 'audioset_classification' already exists and is not an empty directory.


In [0]:
first_run=False